In [7]:
import requests

In [5]:
# load .env and fetch api key
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.environ.get('riot_api_key')

In [49]:
def get_puuid(summonerId:str=None, gameName:str=None, tagLine:str=None, region:str='europe'):
    '''Given summonerId or gameName and tagLine of Riot Account returns the puuid of that account

    Arguments:
        str: summonerId, defualts to None.
        str: gameName, defualts to None.
        str: tagLine, defualts to None.
        str: region, defualts to 'europe'.

    Returns:
        str: puuid
    '''

    if summonerId is not None:
        # region tag is a silly little goose
        if region == 'europe':
            region = 'euw1'
        root = f'https://{region}.api.riotgames.com/'
        endpoint = 'lol/summoner/v4/summoners/'
        request = root + endpoint + summonerId + '?api_key=' + api_key
        response = requests.get(request)
    else:
        if gameName is None or tagLine is None:
            raise Exception("Please input gameName and tagLine")

        request = f'https://{region}.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}?api_key={api_key}'
        response = requests.get(request)

    if not response.status_code == 200:
         raise Exception("Response Error: {}".format(response.status_code))
    puuid = response.json()['puuid']

    return puuid

In [53]:
# testing get_puuid
gameName1 = 'SONDER'
tagLine1 = 'mafk1'
summonerId1 = '0qDuJrMmD3ht2azWwkEZ1iAAuZ3wpWfcrKsTNHe9yIYoEfY'
print("puuid for SONDER#mafk1: " + get_puuid(summonerId=summonerId1))
print("puuid equal using summonerId vs gameName and tagLine?", get_puuid(gameName=gameName1, tagLine=tagLine1) == get_puuid(summonerId=summonerId1))

puuid for SONDER#mafk1: smKXivxW6VCPxTu9NyeWVNNObhayvp7njxu8EmGlOxQ94ttoGDGYp7a9-cgB68kxXUj2EhIWysGfVg
puuid equal using summonerId vs gameName and tagLine? True


In [69]:
def get_ranked_match_history(puuid:str=None, region:str='europe', n_matches:int=20):
    '''Given puuid of Riot Account, returns a list of match IDs as strings

    Arguments:
        str: puuid, defualts to None.
        str: region, defualts to 'europe'.
        int: n_matches, defualts to 20 and must be between 0 and 100 (inclusive).

    Returns:
        list: match history
    '''
    
    if puuid is None:
        raise Exception('Please provide a puuid in string format.')
    if n_matches > 100:
        raise Exception('Amount of matches cannot exceed 100.')
    if n_matches < 0:
        raise Exception('Amount of matches must be at least 0.')

    root = f'https://{region}.api.riotgames.com/'
    endpoint = 'lol/match/v5/matches/by-puuid/'
    request = root + endpoint + puuid + '/ids?type=ranked&start=0&n_matches=' + str(n_matches) + '&api_key=' + api_key
    response = requests.get(request)

    if not response.status_code == 200:
         raise Exception("Response Error: {}".format(response.status_code))

    return response.json()
    


In [68]:
get_ranked_match_history(puuid='smKXivxW6VCPxTu9NyeWVNNObhayvp7njxu8EmGlOxQ94ttoGDGYp7a9-cgB68kxXUj2EhIWysGfVg')

['EUW1_6914529767',
 'EUW1_6914465533',
 'EUW1_6914415568',
 'EUW1_6912180295',
 'EUW1_6912127841',
 'EUW1_6912080895',
 'EUW1_6912002993',
 'EUW1_6911910450',
 'EUW1_6911741762',
 'EUW1_6910380655',
 'EUW1_6910321695',
 'EUW1_6910250611',
 'EUW1_6910165404',
 'EUW1_6910117658',
 'EUW1_6909920044',
 'EUW1_6909864912',
 'EUW1_6909790295',
 'EUW1_6909746745',
 'EUW1_6906674085',
 'EUW1_6906634163']